In [1]:
import sys
sys.path.append("/home/shitijkarsolia/Desktop/venvironments/project1/lib/python3.6/site-packages")

In [2]:
# !pip install utils

# # 

In [3]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import imutils
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
#from utils import LRN2D
import utils


%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.
/home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [4]:
from inception_resnet_v1 import *
model = InceptionResNetV1()

W0913 12:23:02.655467 139649963976512 deprecation_wrapper.py:119] From /home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 12:23:03.130718 139649963976512 deprecation_wrapper.py:119] From /home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 12:23:03.198863 139649963976512 deprecation_wrapper.py:119] From /home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 12:23:03.325808 139649963976512 deprecation_wrapper.py:119] From /home/shitijkarsolia/Desktop/projects/Siamese/snn/lib/python3.6/site-packages/keras/backend/tens

In [5]:
model.load_weights('weights/facenet_weights.h5')
print("weights loaded")

weights loaded


In [6]:
import traceback
def image_to_embedding(image, model):
    try: 
        
        #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
        image = cv2.resize(image, (160, 160)) 
        img = image[...,::-1]
        img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
        x_train = np.array([img])
        embedding = model.predict_on_batch(x_train)
        return embedding
    except:
        traceback.print_exc()
        

In [7]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
        #knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
        #svc = LinearSVC()
        
        #euclidean_distance=np.sum(np.square(embedding-input_embedding))
        #print(type(embedding))
        #print(type(input_embedding))
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
#         a = np.matmul(np.transpose(embedding), input_embedding)
#         b = np.sum(np.multiply(embedding, embedding))
#         c = np.sum(np.multiply(input_embedding, input_embedding))
#         euclidean_distance= 1 - (a / (np.sqrt(b) * np.sqrt(c)))
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 21:
        return str(name)
    else:
        return None

In [ ]:
#This is to make cropped images from the database ( One Time Process)
import glob,cv2,os
import os.path
from os import path
import imutils
import numpy as np
import argparse

# import cv2
prototxt="./deploy.prototxt.txt"
model_det_crop="./res10_300x300_ssd_iter_140000.caffemodel"
# confidence_ref=0.5
net = cv2.dnn.readNetFromCaffe(prototxt, model_det_crop)
count=0
for file in glob.glob("./Employee_Photo/*.jp*g"):
    person_name = os.path.splitext(os.path.basename(file))[0]
    print(person_name)
    print(file)
    image=cv2.imread(file)
# for image in images:
    
#     image = cv2.imread(image)
    count=count+1
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
#     print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.6:
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
#             count=count+1
            # draw the bounding box of the face along with the associated
            # probability
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            #cv2.rectangle(image, (startX, startY), (endX, endY),
              #  (0, 0, 255), 2)
    #         cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
#             cv2.rectangle(image, (startX, startY), (endX, endY),
#                 (0, 0, 255), 2)
    #         cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     

            #cv2.putText(image, text, (startX, y),
            #    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
#             cv2.imwrite("images/User_" + str(count) + ".jpg", image[startY:endY,startX:endX])
            if not path.exists("Employee_Photo/cropped/"+person_name+".jpg"):
                cv2.imwrite("Employee_Photo/cropped/"+person_name+".jpg", image[startY:endY,startX:endX])
            #else:
              #  cv2.imwrite("Employee_Photo/cropped/"+person_name+str(count)+".jpg", image[startY:endY,startX:endX])

In [11]:
#This is to create embeddings from the croppd images in database( One time process)
import glob
import time
import numpy as np
import argparse
import cv2
import traceback
def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("./Employee_Photo/cropped/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        if person_name[0]=='4':
            person_name=person_name.split('_', 1)[-1]
        image_file = cv2.imread(file, 1)
#         face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#         faces = face_detector.detectMultiScale(image_file, 1.3, 5)
#         for (x,y,w,h) in faces:
#             x1 = x
#             y1 = y
#             x2 = x+w
#             y2 = y+h
#             cv2.rectangle(image_file, (x1,y1), (x2,y2), (255,255,255), 2)
#             cv2.imwrite("cropped/"+person_name + ".jpg", image_file[y1:y2,x1:x2])
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    
    prototxt="./deploy.prototxt.txt"
    det_model="./res10_300x300_ssd_iter_140000.caffemodel"
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(1)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    #face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    

# # construct the argument parse and parse the arguments
#     ap = argparse.ArgumentParser()
#    # ap.add_argument("-i", "--image", required=True,
#         #help="path to input image")
# #     ap.add_argument("-p", "--prototxt",default="/home/kamal-keswani/Desktop/FaceRec_Attendence_Quantiphi/Test2_Notebook",
# #         help="path to Caffe 'deploy' prototxt file")
# #     ap.add_argument("-m", "--model", default="/home/kamal-keswani/Desktop/FaceRec_Attendence_Quantiphi/Test2_Notebook",
# #         help="path to Caffe pre-trained model")
#     ap.add_argument("-c", "--confidence", type=float, default=0.5,
#         help="minimum probability to filter weak detections")
#     args = vars(ap.parse_args())
    print("[INFO] loading model...")
    net = cv2.dnn.readNetFromCaffe(prototxt, det_model)
    process_this_frame = True

    while vc.isOpened():
        _, frame = vc.read()
        if process_this_frame:

            img=frame
    #         frame = imutils.resize(frame, width=600)

            # grab the frame dimensions and convert it to a blob
            (h, w) = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                (300, 300), (104.0, 177.0, 123.0))

        # pass the blob through the network and obtain the detections and
        # predictions
            net.setInput(blob)
            detections = net.forward()
            time1= time.time()


            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


            # Loop through all the faces detected 
            identities = []
            for i in range(0, detections.shape[2]):
                # extract the confidence (i.e., probability) associated with the
                # prediction
                confidence = detections[0, 0, i, 2]

                # filter out weak detections by ensuring the `confidence` is
                # greater than the minimum confidence
                if confidence < 0.5:
                    continue

                # compute the (x, y)-coordinates of the bounding box for the
                # object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")



                face_image = frame[max(0, startY):min(h, endY), max(0, startX):min(w, endX)]   
                # face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
                identity = recognize_face(face_image, input_embeddings, model)

                time2= time.time()
                print("Time Taken : ", time2-time1)
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                img=cv2.rectangle(frame, (startX, startY), (endX, endY),
                    (0, 0, 255), 2)
                #img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)

                if identity is not None:
                    cv2.putText(img, str(identity), (startX+5,startY-5), font, 1, (0,0,255), 2)
                    #cv2.putText(frame, text, (startX, y),
                     #   cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    #         frame = imutils.resize(img, width=800)
            key = cv2.waitKey(1)
            cv2.imshow("Face Recognizer", img)
            process_this_frame = not process_this_frame

            if key == 27: # exit on ESC
                break
            
    vc.release()
    cv2.destroyAllWindows()

In [12]:
input_embeddings = create_input_image_embeddings()

In [14]:
recognize_faces_in_cam(input_embeddings)

[INFO] loading model...
Euclidean distance from Nikhil is 11.957331
Euclidean distance from viewPhoto is 12.131477
Euclidean distance from image_resize is 12.581041
Euclidean distance from resize is 12.695586
Euclidean distance from Photo is 14.685105
Euclidean distance from 081744 is 12.924731
Euclidean distance from AJAY KUMAR GOND is 12.195704
Euclidean distance from 0055 is 12.469583
Euclidean distance from Manideepa_Pabbathineni(Bangalore) is 11.07665
Euclidean distance from size is 12.763551
Euclidean distance from gautham is 12.786523
Euclidean distance from 200 is 10.656072
Euclidean distance from kristen is 11.177082
Euclidean distance from DSC_6418 (2) is 11.634967
Euclidean distance from Image is 12.551946
Euclidean distance from Bengaluru is 12.548346
Euclidean distance from IMG-20190610-WA0006 is 10.985185
Euclidean distance from Formal Photo is 12.674807
Euclidean distance from KAMAKSHI ANANTHARAMAN  is 12.628353
Euclidean distance from IMG_20190715_191550 is 13.513538
Eu

Euclidean distance from Nikhil is 11.793448
Euclidean distance from viewPhoto is 12.264102
Euclidean distance from image_resize is 12.736155
Euclidean distance from resize is 12.753648
Euclidean distance from Photo is 14.571985
Euclidean distance from 081744 is 12.888428
Euclidean distance from AJAY KUMAR GOND is 12.2590065
Euclidean distance from 0055 is 12.471306
Euclidean distance from Manideepa_Pabbathineni(Bangalore) is 11.062571
Euclidean distance from size is 12.803204
Euclidean distance from gautham is 12.683151
Euclidean distance from 200 is 10.817917
Euclidean distance from kristen is 11.066763
Euclidean distance from DSC_6418 (2) is 11.7562275
Euclidean distance from Image is 12.649049
Euclidean distance from Bengaluru is 12.547546
Euclidean distance from IMG-20190610-WA0006 is 11.062712
Euclidean distance from Formal Photo is 12.618402
Euclidean distance from KAMAKSHI ANANTHARAMAN  is 12.722189
Euclidean distance from IMG_20190715_191550 is 13.630164
Euclidean distance from

Euclidean distance from PicsArt_06-15-12.55.51 is 11.628884
Euclidean distance from 17469 is 12.083081
Euclidean distance from 7497 is 11.67496
Euclidean distance from pradeep is 12.397331
Euclidean distance from 21 is 12.146609
Euclidean distance from Mrutyunjay Hiremath is 12.141988
Euclidean distance from 194533 is 12.047563
Euclidean distance from Sohail Anwar- Bangalore is 11.50588
Euclidean distance from IMG-5871 is 12.454804
Euclidean distance from Shivani is 13.068761
Euclidean distance from scan0044 is 10.895885
Euclidean distance from 141456B4-CB2D-464A-AB0A-094B980DC0E3 is 12.246878
Euclidean distance from Piyush Profile Pic is 11.794065
Euclidean distance from New Doc 2018-10-01 18.22.44 (7) is 11.506337
Euclidean distance from 807 is 13.120037
Euclidean distance from Maitri_Fafadia_PhotoGraph_DJSCE is 10.985475
Euclidean distance from New Doc 2018-10-10 12.30.31 (8) is 12.52586
Euclidean distance from 16 is 11.248212
Euclidean distance from Felicia Photograph is 13.569718


Euclidean distance from Tanmay_Parulekar is 12.482972
Euclidean distance from 4 is 12.16223
Euclidean distance from preview 1 is 12.00251
Euclidean distance from New Doc 2018-10-10 12.30.31(4) is 13.040927
Euclidean distance from PicsArt_06-15-12.55.51 is 11.7880745
Euclidean distance from 17469 is 12.72039
Euclidean distance from 7497 is 12.569627
Euclidean distance from pradeep is 12.317711
Euclidean distance from 21 is 12.595208
Euclidean distance from Mrutyunjay Hiremath is 12.748741
Euclidean distance from 194533 is 12.293431
Euclidean distance from Sohail Anwar- Bangalore is 12.11694
Euclidean distance from IMG-5871 is 12.800025
Euclidean distance from Shivani is 13.041284
Euclidean distance from scan0044 is 11.350782
Euclidean distance from 141456B4-CB2D-464A-AB0A-094B980DC0E3 is 12.610395
Euclidean distance from Piyush Profile Pic is 12.244504
Euclidean distance from New Doc 2018-10-01 18.22.44 (7) is 12.112726
Euclidean distance from 807 is 13.610858
Euclidean distance from Ma

Euclidean distance from Nikhil is 11.867147
Euclidean distance from viewPhoto is 12.029176
Euclidean distance from image_resize is 12.509524
Euclidean distance from resize is 12.459463
Euclidean distance from Photo is 14.41394
Euclidean distance from 081744 is 12.794466
Euclidean distance from AJAY KUMAR GOND is 12.128109
Euclidean distance from 0055 is 12.433447
Euclidean distance from Manideepa_Pabbathineni(Bangalore) is 10.953535
Euclidean distance from size is 12.795764
Euclidean distance from gautham is 12.609995
Euclidean distance from 200 is 10.480522
Euclidean distance from kristen is 11.125102
Euclidean distance from DSC_6418 (2) is 11.448162
Euclidean distance from Image is 12.441955
Euclidean distance from Bengaluru is 12.405052
Euclidean distance from IMG-20190610-WA0006 is 10.769511
Euclidean distance from Formal Photo is 12.508015
Euclidean distance from KAMAKSHI ANANTHARAMAN  is 12.546936
Euclidean distance from IMG_20190715_191550 is 13.385521
Euclidean distance from IM

...

Euclidean distance from IIFT is 14.3321705
Euclidean distance from 161615 is 14.189943
Euclidean distance from 111717 is 15.183634
Euclidean distance from Ketav is 14.008101
Euclidean distance from Gaurav Maniyar is 12.835879
Euclidean distance from ipg_2014074 is 16.466675
Euclidean distance from bangalore is 13.997803
Euclidean distance from New Doc 2018-10-10 12.30.31 (3) is 15.39557
Euclidean distance from Gail Cardozo is 15.581592
Euclidean distance from 103518 is 14.503891
Euclidean distance from New Doc 2019-07-03 12.15.08 is 14.309692
Euclidean distance from Vasant_Mahadik_Passport Size Photo is 12.848977
Euclidean distance from Hitesh(Bangalore) is 14.224968
Euclidean distance from Vasu(Bangalore) is 15.446726
Euclidean distance from 182940 is 14.8623495
Euclidean distance from IMG_20190716_150042 is 12.429665
Euclidean distance from Passport Photo (2019.03.31) - Rijul Raut is 14.060271
Euclidean distance from card is 10.854781
Euclidean distance from 200101 is 13.385283
Eucli

Euclidean distance from Nikhil is 11.976783
Euclidean distance from viewPhoto is 11.119056
Euclidean distance from image_resize is 12.08467
Euclidean distance from resize is 12.97478
Euclidean distance from Photo is 14.622597
Euclidean distance from 081744 is 13.008891
Euclidean distance from AJAY KUMAR GOND is 12.03869
Euclidean distance from 0055 is 13.4067955
Euclidean distance from Manideepa_Pabbathineni(Bangalore) is 11.857363
Euclidean distance from size is 11.748518
Euclidean distance from gautham is 12.976201
Euclidean distance from 200 is 10.690718
Euclidean distance from kristen is 11.626175
Euclidean distance from DSC_6418 (2) is 11.615712
Euclidean distance from Image is 13.861104
Euclidean distance from Bengaluru is 13.072489
Euclidean distance from IMG-20190610-WA0006 is 12.649966
Euclidean distance from Formal Photo is 13.216002
Euclidean distance from KAMAKSHI ANANTHARAMAN  is 13.633199
Euclidean distance from IMG_20190715_191550 is 13.894282
Euclidean distance from IMG

Euclidean distance from jogdand is 12.522537
Euclidean distance from 399 is 13.180197
Euclidean distance from V Subbaiah Bangalore is 12.68152
Euclidean distance from DSC_0145 is 10.8279085
Euclidean distance from Mustufahasan Shabbir Surti is 14.707903
Euclidean distance from Shobha is 12.657757
Euclidean distance from 15040-115 is 12.045247
Euclidean distance from IMG-20190507-WA0002 is 13.297594
Euclidean distance from risheekesh is 13.630929
Euclidean distance from New Doc 2018-10-10 12.30.31 (2) is 12.6396
Euclidean distance from Disha_Shetty_Photo_PCCOE is 12.443485
Euclidean distance from PP is 11.742008
Euclidean distance from PhotoGrid_1562071481388 is 12.260231
Euclidean distance from Ketan S Palankar Bangalore is 12.453218
Euclidean distance from whiteback is 11.839358
Euclidean distance from WhatsApp Image 2019-06-08 at 17.52.25 is 11.919473
Euclidean distance from IMG_20190703_105824 is 12.509686
Euclidean distance from 0007 - Copy is 12.390396
Euclidean distance from Parv

Euclidean distance from Nikhil is 11.843772
Euclidean distance from viewPhoto is 12.216018
Euclidean distance from image_resize is 12.871857
Euclidean distance from resize is 12.543742
Euclidean distance from Photo is 14.66291
Euclidean distance from 081744 is 12.647206
Euclidean distance from AJAY KUMAR GOND is 12.175121
Euclidean distance from 0055 is 12.349734
Euclidean distance from Manideepa_Pabbathineni(Bangalore) is 10.995884
Euclidean distance from size is 12.634382
Euclidean distance from gautham is 12.960284
Euclidean distance from 200 is 10.359374
Euclidean distance from kristen is 10.742418
Euclidean distance from DSC_6418 (2) is 11.562021
Euclidean distance from Image is 12.795501
Euclidean distance from Bengaluru is 12.46831
Euclidean distance from IMG-20190610-WA0006 is 10.963075
Euclidean distance from Formal Photo is 12.666425
Euclidean distance from KAMAKSHI ANANTHARAMAN  is 12.527324
Euclidean distance from IMG_20190715_191550 is 13.454047
Euclidean distance from IMG

Euclidean distance from Sai Appala Raju I (Banglore) is 11.998063
Euclidean distance from kavish mehta(Bangalore) is 11.946393
Euclidean distance from pereira is 13.568399
Euclidean distance from Akash is 12.204543
Euclidean distance from Photo ID white BG is 13.298794
Euclidean distance from cat-baby-khushboo-sahu is 11.708792
Euclidean distance from WhatsApp Image 2019-08-28 at 10.53.50 AM is 11.993962
Euclidean distance from IIFT is 11.806878
Euclidean distance from 161615 is 12.115869
Euclidean distance from 111717 is 13.043682
Euclidean distance from Ketav is 11.967447
Euclidean distance from Gaurav Maniyar is 12.993451
Euclidean distance from ipg_2014074 is 12.475115
Euclidean distance from bangalore is 11.794595
Euclidean distance from New Doc 2018-10-10 12.30.31 (3) is 13.091106
Euclidean distance from Gail Cardozo is 12.1540165
Euclidean distance from 103518 is 12.321712
Euclidean distance from New Doc 2019-07-03 12.15.08 is 12.640267
Euclidean distance from Vasant_Mahadik_Pas